In [ ]:
!pip install transformers datasets accelerate
!pip install groq

In [ ]:
with open('10.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

In [ ]:
def split_text_into_chunks(text, max_chunk_chars):
    import textwrap
    chunks = textwrap.wrap(text, max_chunk_chars, break_long_words=False, break_on_hyphens=False)
    print(f"Total chunks created: {len(chunks)}\n")
    print("Sample chunk: ")
    print(chunks[8])
    return chunks

chunks = split_text_into_chunks(full_text, max_chunk_chars=2000)

In [ ]:
from groq import Groq
import time

client = Groq(api_key= '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')  # Replace with your key

qa_pairs = []

for idx, chunk in enumerate(chunks):  # Adjust chunk limit as needed
    print(f"Processing chunk {idx + 1} of {len(chunks)}")
    # Step 4.1: Generate 5 questions from this chunk
    try:
        messages = [
            {"role": "system", "content": "You are being used to generate a dataset for finrtuning a llm a Indian GST law chatbot."},
            {"role": "user", "content": f"Generate 5 unique and insightful questions based on the following legal text that a user may ask about Indian GST law:\n\n{chunk}\n The question should look like a query posed by a user to the GST chatbot. Do not  use meta-explanations (e.g., This question requires the chatbot to...) as the question."}
        ]
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=messages,
            temperature=0.7,
            max_tokens=400
        )
        questions_text = response.choices[0].message.content.strip()
        questions = [q.strip("-•1234567890. ").strip() for q in questions_text.split("\n") if q.strip()]
        questions = questions[1:]  # Keep only the first 5 questions
    except Exception as e:
        print(f"[QGen Error] at chunk {idx}: {e}")
        continue

    # Step 4.2: Get answers for each question
    for question in questions:
        try:
            messages = [
                {"role": "system", "content": "I want you to answer the following question on Indian GST law to create answers for a dataset for finetuning an llm. just provide th answer, do not include any comments like 'based on the context provided', just give the outright answer. Do not give answers that are too short or vague. Avoid inconsistent use of \n for bullet points unless you're using proper markdown formatting throughout."},
                {"role": "user", "content": f"Question: {question}\n\nRefer to this legal content:\n{chunk}"}
            ]
            response = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=messages,
                temperature=0.7,
                max_tokens=300
            )
            answer = response.choices[0].message.content.strip()
            qa_pairs.append({
                "question": question,
                "answer": answer
            })
            time.sleep(0.3)
        except Exception as e:
            print(f"[Answer Error] for question '{question[:30]}': {e}")

In [ ]:
# STEP 5: Create and Save the Dataset
from datasets import Dataset

dataset = Dataset.from_list(qa_pairs)
dataset.to_json("output10.json")

# STEP 6: Download the file
# files.download("gst_qa_dataset.json")

In [ ]:
from google.colab import files

files.download("output10.json")